# RAG関連手法の学習 - 3. Query Construction
* https://python.langchain.com/docs/concepts/#retrieval

In [1]:
from dotenv import load_dotenv
import os
from PIL import Image
# 環境変数読み込み
load_dotenv()

# 画像表示関数
def show_image(path):
    im = Image.open(path)
    return im

## 3. Query Constuction
Using an LLM to convert a natural language query into a query syntax is a popular and powerful approach.

1. **Text to SQL**
    * If users are asking questions that require information housed in a relational database, accessible via SQL.
    * This uses an LLM to transform user input into a SQL query.
    * https://python.langchain.com/docs/tutorials/sql_qa/
2. **Text-to-Cypher**
    * If users are asking questions that require information housed in a graph database, accessible via Cypher.
    * This uses an LLM to transform user input into a Cypher query.
    * https://python.langchain.com/docs/tutorials/graph/
3. **Self Query**
    * If users are asking questions that are better answered by fectching documents based on metadata rather than similarity with the text.
    * This uses an LLM to transform user input into two things: (1) a string to look up semantically, (2) a metadata filter to go along with it. This is useful because oftentimes questions are about the MEATADATA of documents (not the content itself).
    * https://python.langchain.com/docs/how_to/self_query/



### 3-1. Text to SQL
* https://python.langchain.com/docs/tutorials/sql_qa/

* Enabling a LLM system to query structured data can be qualitatively different from unstructured text data. Whereas in the latter it is common to generate text that can be searched against a vector database, the approach for structured data is often for the LLM to write and execute queries in a DSL, such as SQL. In this guide we'll go over the basic ways to create a Q&A system over tabular data in databases. We will cover implementations using both chains and agents. These systems will allow us to ask a question about the data in a database and get back a natural language answer. The main difference between the two is that our agent can query the database in a loop as many times as it needs to answer the question.

#### Architecture
At a high-level, the steps of these systems are:

1. Convert question to DSL query: Model converts user input to a SQL query.
2. Execute SQL query: Execute the query.
3. Answer the question: Model responds to user input using the query results.

In [38]:
### prepare SQL Database

from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:////Users/user/for_IT/learn_rag/data/chinook.db")
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'tracks']


In [39]:
db.run("SELECT * FROM artists LIMIT 10;")

"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

#### Chains
1. convert the question into a SQL query
2. execute the query
3. use the result to answer the original question

###### 1. convert the question into a SQL query


In [40]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


from langchain.chains import create_sql_query_chain
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there"})
response

'SQLQuery: SELECT COUNT("EmployeeId") AS "EmployeeCount" FROM employees;'

##### 2. execute the query

Now that we've generated a SQL query, we'll want to execute it. **This is the most dangerous part of creating a SQL chain**. Consider carefully if it is OK to run automated queries over your data. Minimize the database connection permissions as much as possible. Consider adding a human approval step to you chains before query execution (see below).

We can use the QuerySQLDatabaseTool to easily add query execution to our chain:

In [47]:
### 上記では、SQL文に余計な文字が入るので、以下のように、整形するtemplateを追加する。
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


from langchain.chains import create_sql_query_chain
from langchain.prompts import ChatPromptTemplate
sql_mod_tmt = ChatPromptTemplate.from_template(
    """
    You are a great database engineer. Please make SQL for query.
    Answer is just SQL only and don't add any word expect for SQL like following example.
    error: 'sql\nSELECT A fFROM table;'
    correct: 'SELECT A fFROM table;'

    Question: {input}
    """
)

chain = create_sql_query_chain(llm, db) | sql_mod_tmt | llm
response = chain.invoke ({"question": "How many employees are there"})
response

AIMessage(content='SELECT COUNT("EmployeeId") AS "EmployeeCount" FROM employees;', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 85, 'total_tokens': 99, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-c56c2395-42d4-43c4-b28a-8b322d7875db-0', usage_metadata={'input_tokens': 85, 'output_tokens': 14, 'total_tokens': 99, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [49]:
print(response.content)

SELECT COUNT("EmployeeId") AS "EmployeeCount" FROM employees;


In [61]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.runnables import RunnableLambda

# SQLクエリ実行用
execute_query = QuerySQLDataBaseTool(db=db)
# AIMessageからSQLクエリを抽出するランナブル
extract_sql_query = RunnableLambda(lambda x: x.content)

# SQL作成と検索用chain
chain = (create_sql_query_chain(llm, db)
         | sql_mod_tmt
         | llm
         | extract_sql_query
         | execute_query
)
# 実行
result = chain.invoke({"question": "How many employees are there"})

print(result)

[(8,)]


##### 3. answer the question


In [67]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

write_query = (create_sql_query_chain(llm, db)
         | sql_mod_tmt
         | llm
         | extract_sql_query
)

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

final_chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

final_chain.invoke({"question": "How many employees are there"})

'There are 8 employees.'

### 3-2. Text-to-Cypher
* https://python.langchain.com/docs/tutorials/graph/